In [21]:
import math

def gmpe_reso_14(mag, dist, mode='pga', vs30=800, sof='sofN'):
    '''Gmpe Bindi et al 2014 Rjb
    
    :param mag: float , magnitude
    :param dist: float, distance in km
    :param mode: string, optional: either 'pga' (the default) or 'pgv'
    :param vs30: float, optional: the vs30 (defaults to 800)
    :param sof: string, optional: the style of faulting, either 'sofN' (the default), 'sofR', sofS'
    
    :return: The float representing the gmpe's pgv or pga (depending on `mode`), in m/sec or m/sec^2, respectively
    '''
    if mode == 'pgv':
        
        #     imt             e1             c1            c2             h            c3             b1             b2            b3          gamma           sofN           sofR           sofS           tau           phi        phis2s         sigma
        #     pgv    2.264810000   -1.224080000   0.202085000   5.061240000   0.000000000    0.162802000   -0.092632400   0.044030100   -0.529443000   -0.009476750    0.040057400   -0.030580500   0.156062000   0.277714000   0.120398000   0.318560000

        
        e1, c1, c2, h, c3, b1, b2, b3, sA = (2.264810000,
                                             -1.224080000,
                                             0.202085000,
                                             5.061240000,
                                             0.0,
                                             0.162802000,
                                             -0.092632400,
                                             0.044030100,
                                             -0.529443000)
        if sof == 'sofR':
            sof_val = 0.040057400
        elif sof == 'sofS':
            sof_val = -0.030580500
        else:
            sof_val = -0.009476750
        
    else:
        
        #     imt             e1             c1            c2             h            c3             b1             b2            b3          gamma           sofN           sofR           sofS           tau           phi        phis2s         sigma
        #     pga    3.328190000   -1.239800000   0.217320000   5.264860000   0.001186240   -0.085504500   -0.092563900   0.000000000   -0.301899000   -0.039769500    0.077525300   -0.037755800   0.149977000   0.282398000   0.165611000   0.319753000

        
        e1, c1, c2, h, c3, b1, b2, b3, sA = (3.328190000,
                                             -1.239800000,
                                             0.217320000,
                                             5.264860000,
                                             0.001186240,
                                             -0.085504500,
                                             -0.092563900,
                                             0.000000000,
                                             -0.301899000)
        if sof == 'sofR':
            sof_val = 0.077525300
        elif sof == 'sofS':
            sof_val = -0.037755800
        else:
            sof_val = -0.039769500

    Mh = 6.75
    Mref = 5.5
    VREF = 800
    
    exprM1 = mag - Mh
    exprD1 = mag - Mref
  
    exprM3 = b1 * exprM1 + b2 * exprM1 ** 2
    exprM4 = b3 * exprM1
    valueFM = exprM3 if mag <= Mh else exprM4  # valueFM <- ifelse(MAG<=.Mh, .exprM3, .exprM4)
  
    Rref = 1.0
    exprD2 = c1 + c2 * exprD1  # [c1 + c2(M-Mref)]
    exprD3 = (dist ** 2 + h ** 2)  # [Rjb^2 + h^2]
    exprD4 = math.sqrt(exprD3)  # [sqrt[Rjb^2 + h^2]]
    exprD5 = exprD4 / Rref      # [sqrt[Rjb^2 + h^2]/Rref]
    exprD6 = math.log10(exprD5)   # LN[sqrt[Rjb^2 + h^2]/Rref]
    exprD8 = exprD4 - Rref  # [sqrt[Rjb^2 + h^2] - Rref]
    valueFD = exprD2 * exprD6 - c3 * exprD8
    
    valueFS = sA * math.log10(vs30 / VREF)

    ## Value ##
    return 10 ** (e1 + valueFD + valueFM + valueFS + sof_val) / 100.0  # returns m/sec or m/sec2

In [23]:
import numpy as np
from scipy.constants import g
assert np.isclose(gmpe_reso_14(6, 200, 'pga', sof='sofN'), 0.02893213698593125)
# assert g value is the same:
assert np.isclose(gmpe_reso_14(6, 200, 'pga', sof='sofN')/g, 0.002949594, rtol=1e-3)
assert np.isclose(gmpe_reso_14(6, 200, 'pgv'), 0.0031392574204313233)
print('OK, test passed')

OK, test passed


In [12]:
# original R code (above, we changed the coefficients and added style of faulting options)

# pga_pre, pgv_pre, pga_obs_, pgv_obs, f0.5_obs f1_obs f2_obs f5_obs f10_obs (in Hertz)
# f0.5_pre f1_pre f2_pre f5_pre f10_pre (in Hertz, for the moment empty)
# Mag, dist, snr, saturated


# pgvReso14 <- function(MAG,DIST) #bindi et al 2014 Hypo PGV
# {
#   VS30 <- 800
#   e1<- 3.24249 
  
#   b1<- 0.472433
#   b2<- -0.072548
#   b3<- 0.436952
  
#   c1<- -1.57556
#   c2<- 0.0791774
#   c3<- 0.0
#   sA<- -0.508833
  
#   h<- 4.38918
  
#   .Mh<- 6.75
#   .Mref<- 5.5
  
#   .exprM1 <- MAG - .Mh
#   .exprD1 <- MAG - .Mref
  
#   .exprM3 <- b1 * .exprM1 + b2 * .exprM1 ^ 2
#   .exprM4 <- b3 * .exprM1 
#   .valueFM <- ifelse(MAG<=.Mh, .exprM3, .exprM4)
  
#   .Rref <- 1.0
#   .exprD2 <- c1 + c2 * .exprD1  # [c1 + c2(M-Mref)]
#   .exprD3 <- (DIST^2+h^2)  # [Rjb^2 + h^2]
#   .exprD4 <- sqrt(.exprD3)  # [sqrt[Rjb^2 + h^2]]
#   .exprD5 <- .exprD4/.Rref      # [sqrt[Rjb^2 + h^2]/Rref]
#   .exprD6 <- log10(.exprD5)   # LN[sqrt[Rjb^2 + h^2]/Rref]
#   .exprD8 <- .exprD4 - .Rref  # [sqrt[Rjb^2 + h^2] - Rref]
#   .valueFD <- .exprD2 * .exprD6 - c3 * .exprD8
  
#   VREF <- 800
#   .valueFS <- sA*log10(VS30/VREF)
  
#   ## Value ##
#   .value <- 0.01* 10^(e1 + .valueFD + .valueFM + .valueFS)
#   return(.value)
# }
# ##
# pgaReso14 <- function(MAG,DIST) #bindi ey al 2014 Hypo PGV
# {
#   VS30 <- 800
#   e1<- 4.27391 
  
#   b1<- 0.217109
#   b2<- -0.068256
#   b3<- 0.352976
  
#   c1<- -1.57821
#   c2<- 0.1082184
#   c3<- 0.0
#   sA<- -0.293242
  
#   h<- 4.82743
  
#   .Mh<- 6.75
#   .Mref<- 5.5
  
#   .exprM1 <- MAG - .Mh
#   .exprD1 <- MAG - .Mref
  
#   .exprM3 <- b1 * .exprM1 + b2 * .exprM1 ^ 2
#   .exprM4 <- b3 * .exprM1 
#   .valueFM <- ifelse(MAG<=.Mh, .exprM3, .exprM4)
  
#   .Rref <- 1.0
#   .exprD2 <- c1 + c2 * .exprD1  # [c1 + c2(M-Mref)]
#   .exprD3 <- (DIST^2+h^2)  # [Rjb^2 + h^2]
#   .exprD4 <- sqrt(.exprD3)  # [sqrt[Rjb^2 + h^2]]
#   .exprD5 <- .exprD4/.Rref      # [sqrt[Rjb^2 + h^2]/Rref]
#   .exprD6 <- log10(.exprD5)   # LN[sqrt[Rjb^2 + h^2]/Rref]
#   .exprD8 <- .exprD4 - .Rref  # [sqrt[Rjb^2 + h^2] - Rref]prova1
#   .valueFD <- .exprD2 * .exprD6 - c3 * .exprD8
  
#   VREF <- 800
#   .valueFS <- sA*log10(VS30/VREF)
  
#   ## Value ##
#   .value <- 0.01 *10^(e1 + .valueFD + .valueFM + .valueFS)
#   return(.value)
# }


0.0002949254841997961
